# QoE Prediction Model Building


## 1. Compare Models in Federeted Learning Scenerio


Import libraries:


In [1]:
from keras import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from keras.metrics import MeanSquaredError
from typing import List, Tuple
from numpy.typing import NDArray
from functools import reduce
from numpy import add
import pandas as pd
import json
import matplotlib.pyplot as plt

# weights
Layers = List[NDArray]
# weights and number of sample
ModelUpdate = Tuple[Layers, int]

2023-07-10 17:21:29.513287: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-10 17:21:29.550178: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-10 17:21:29.737308: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-10 17:21:29.739036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 17:21:30.846236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Check if Tensorflow can detect GPU:


In [2]:
import tensorflow as tf

tf.config.list_physical_devices()

2023-07-09 00:01:27.145577: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-09 00:01:27.289013: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

Read dataset from file and split to samples and target:


In [2]:
def read_data(file_id) -> Tuple[pd.DataFrame, pd.Series]:
    df = pd.read_csv(
        "./data/pokemon/chunk-{}.csv".format(file_id), index_col="id"
    ).drop(["user_id"], axis=1)
    x = df.drop(["MOS"], axis=1)
    y = df["MOS"]
    return x, y

Prepare data for models initialization and evaluation:


In [3]:
x_init, y_init = read_data("init")
x_test, y_test = read_data("test")

In [7]:
x_init

,QoA_VLCresolution,QoA_VLCbitrate,QoA_VLCframerate,QoA_VLCdropped,QoA_VLCaudiorate,QoA_VLCaudioloss,QoA_BUFFERINGcount,QoA_BUFFERINGtime,QoS_type,QoS_operator,QoU_sex,QoU_age,QoU_Ustedy,QoF_begin,QoF_shift,QoF_audio,QoF_video
id,,,,,,,,,,,,,,,,,
1630,360,93.661070,12.083333,0,21.816667,0,2,1455,5,4,1,33,5,5,5,3,3
1602,360,74.690620,8.300000,0,21.833333,0,1,2885,5,4,1,33,5,3,5,3,3
1628,360,63.705845,12.250000,1,22.283333,1,1,2042,5,4,1,33,5,5,5,1,4
1625,360,629.918800,25.400000,0,44.450000,0,1,1699,5,4,1,33,5,5,5,4,5
1600,360,97.613490,11.933333,0,21.833333,0,2,2099,5,4,1,33,5,4,5,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,360,435.797200,24.966667,0,44.133333,0,2,2216,2,3,1,30,5,3,5,4,4
1730,360,109.510414,11.933333,0,21.833333,0,1,2708,4,4,1,48,5,3,5,2,4
1758,360,566.690500,24.666667,0,43.783333,0,1,3136,4,3,1,48,5,4,5,4,4


Define RNN models:


In [6]:
def lstm() -> Sequential:
    model = Sequential(
        layers=[
            LSTM(128, input_shape=(17, 1)),
            Dense(128, activation="relu"),
            Dropout(0.2),
            Dense(64),
            Dropout(0.2),
            Dense(32),
            Dropout(0.2),
            Dense(1, activation="linear"),
        ]
    )
    model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])
    return model


def bidirectional_lstm():
    model = Sequential(
        layers=[
            Bidirectional(LSTM(128), input_shape=(17, 1)),
            Dense(128, activation="relu"),
            Dropout(0.2),
            Dense(64),
            Dropout(0.2),
            Dense(32),
            Dropout(0.2),
            Dense(1, activation="linear"),
        ]
    )
    model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])
    return model


def gru():
    model = Sequential(
        layers=[
            GRU(128, input_shape=(17, 1)),
            Dense(128, activation="relu"),
            Dropout(0.2),
            Dense(64),
            Dropout(0.2),
            Dense(32),
            Dropout(0.2),
            Dense(1, activation="linear"),
        ]
    )
    model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])
    return model

Utilities functions for train:


In [7]:
def train(
    model: Sequential, weights: Layers, samples: pd.DataFrame, targets: pd.Series
) -> ModelUpdate:
    model.set_weights(weights)

    model.fit(
        samples, targets, validation_split=0.05, batch_size=128, epochs=20, shuffle=True
    )

    return model.get_weights(), len(samples)

In [8]:
def fed_avg(updates: List[ModelUpdate]) -> Layers:
    num_examples_total = sum([num_examples for _, num_examples in updates])

    weights = [
        [layer * num_examples for layer in layers] for layers, num_examples in updates
    ]

    return [
        reduce(add, layer_updates) / num_examples_total
        for layer_updates in zip(*weights)
    ]

Init models with some data:


In [9]:
fl_models = [lstm(), bidirectional_lstm(), gru()]
model_names = ["LSTM", "Bidirectional LSTM", "GRU"]
init_weights = [
    train(model, model.get_weights(), x_init, y_init)[0] for model in fl_models
]
fl_result = {}
for name in model_names:
    fl_result[name] = []

2023-06-19 13:57:57.044448: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-19 13:57:57.046154: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-19 13:57:57.046972: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/20


2023-06-19 13:57:57.760944: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-19 13:57:57.762690: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-19 13:57:57.763762: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 2s 2s/step - loss: 10.7881 - mean_squared_error: 10.7881 - val_loss: 9.3657 - val_mean_squared_error: 9.3657
Epoch 2/20
1/1 [==============================] - 0s 41ms/step - loss: 7.0557 - mean_squared_error: 7.0557 - val_loss: 5.7276 - val_mean_squared_error: 5.7276
Epoch 3/20


2023-06-19 13:57:59.212875: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-19 13:57:59.214586: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-19 13:57:59.215579: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

...

1/1 [==============================] - 3s 3s/step - loss: 8.4575 - mean_squared_error: 8.4575 - val_loss: 3.3047 - val_mean_squared_error: 3.3047
Epoch 2/20
1/1 [==============================] - 0s 39ms/step - loss: 2.8255 - mean_squared_error: 2.8255 - val_loss: 0.5894 - val_mean_squared_error: 0.5894
Epoch 3/20
1/1 [==============================] - 0s 40ms/step - loss: 2.1906 - mean_squared_error: 2.1906 - val_loss: 0.8180 - val_mean_squared_error: 0.8180
Epoch 4/20
1/1 [==============================] - 0s 39ms/step - loss: 3.2067 - mean_squared_error: 3.2067 - val_loss: 0.8468 - val_mean_squared_error: 0.8468
Epoch 5/20
1/1 [==============================] - 0s 39ms/step - loss: 3.1333 - mean_squared_error: 3.1333 - val_loss: 0.5748 - val_mean_squared_error: 0.5748
Epoch 6/20
1/1 [==============================] - 0s 40ms/step - loss: 2.1055 - mean_squared_error: 2.1055 - val_loss: 0.5529 - val_mean_squared_error: 0.5529
Epoch 7/20
1/1 [==============================] - 0s 38ms/s

2023-06-19 13:58:03.602822: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-19 13:58:03.603762: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-19 13:58:03.605344: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 2s 2s/step - loss: 12.2695 - mean_squared_error: 12.2695 - val_loss: 10.4825 - val_mean_squared_error: 10.4825
Epoch 2/20
1/1 [==============================] - 0s 38ms/step - loss: 8.2630 - mean_squared_error: 8.2630 - val_loss: 6.7738 - val_mean_squared_error: 6.7738
Epoch 3/20
1/1 [==============================] - 0s 37ms/step - loss: 5.3289 - mean_squared_error: 5.3289 - val_loss: 3.8188 - val_mean_squared_error: 3.8188


2023-06-19 13:58:04.896630: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-19 13:58:04.897776: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-19 13:58:04.898836: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 4/20
1/1 [==============================] - 0s 38ms/step - loss: 3.3927 - mean_squared_error: 3.3927 - val_loss: 1.6454 - val_mean_squared_error: 1.6454
Epoch 5/20
1/1 [==============================] - 0s 44ms/step - loss: 1.4602 - mean_squared_error: 1.4602 - val_loss: 0.5377 - val_mean_squared_error: 0.5377
Epoch 6/20
1/1 [==============================] - 0s 38ms/step - loss: 1.3894 - mean_squared_error: 1.3894 - val_loss: 0.5119 - val_mean_squared_error: 0.5119
Epoch 7/20
1/1 [==============================] - 0s 37ms/step - loss: 1.7280 - mean_squared_error: 1.7280 - val_loss: 0.8327 - val_mean_squared_error: 0.8327
Epoch 8/20
1/1 [==============================] - 0s 34ms/step - loss: 1.8192 - mean_squared_error: 1.8192 - val_loss: 0.9334 - val_mean_squared_error: 0.9334
Epoch 9/20
1/1 [==============================] - 0s 35ms/step - loss: 2.6926 - mean_squared_error: 2.6926 - val_loss: 0.6991 - val_mean_squared_error: 0.6991
Epoch 10/20
1/1 [=============================

In each round, each model will be trained with 25 dataset separately, then 25 sets of weights will be aggregated


In [ ]:
for round in range(6):
    datasets = [read_data(25 * round + i) for i in range(1, 26)]

    for name, model in zip(model_names, fl_models):
        old_weights = model.get_weights()
        local_updates = [train(model, old_weights, x, y) for (x, y) in datasets]
        new_global_weights = fed_avg(local_updates)
        model.set_weights(new_global_weights)
        result = model.evaluate(x_test, y_test)[0]
        fl_result[name].append(result)

In [ ]:
json_result = json.dumps(fl_result, indent=4)
print(json_result)

In [ ]:
plt.plot(fl_result["LSTM"], "o-r", label="lstm")
plt.plot(fl_result["Bidirectional LSTM"], "o-b", label="bidirectional lstm")
plt.plot(fl_result["GRU"], "o-g", label="gru")

x = [25 * i for i in range(1, 7)]
xi = list(range(len(x)))
plt.xticks(xi, x)

plt.xlabel("Number of Client")
plt.ylabel("MSE")

plt.legend()
plt.show()

In [ ]:
cl_model = lstm()
cl_model.set_weights(init_weights[0])

In [ ]:
cl_result = []

In [ ]:
for round in range(6):
    datasets = [read_data(i) for i in range(1, 25 * (round + 1))]
    x = pd.concat([x for x, _ in datasets], axis=0)
    y = pd.concat([y for _, y in datasets], axis=0)
    train(cl_model, init_weights[0], x, y)
    result = cl_model.evaluate(x_test, y_test)[0]
    cl_result.append(result)

In [ ]:
plt.plot(cl_result, "o-r", label="Central")
plt.plot(fl_result["LSTM"], "o-b", label="FedAvg")
x = [25 * i for i in range(1, 7)]
xi = list(range(len(x)))
plt.xticks(xi, x)

plt.xlabel("Number of Client")
plt.ylabel("MSE")

plt.legend()
plt.show()

In [9]:
model = lstm()

2023-07-09 00:02:24.137759: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-09 00:02:24.139588: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-09 00:02:24.140683: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [10]:
train(model, model.get_weights(), x_init, y_init)

Epoch 1/20


2023-07-09 00:02:28.829363: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-09 00:02:28.830963: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-09 00:02:28.831956: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 2s 2s/step - loss: 10.0584 - mean_squared_error: 10.0584 - val_loss: 7.7985 - val_mean_squared_error: 7.7985
Epoch 2/20
1/1 [==============================] - 0s 41ms/step - loss: 6.0092 - mean_squared_error: 6.0092 - val_loss: 4.0568 - val_mean_squared_error: 4.0568
Epoch 3/20


2023-07-09 00:02:30.424302: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-09 00:02:30.426059: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-09 00:02:30.426801: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 0s 46ms/step - loss: 3.2565 - mean_squared_error: 3.2565 - val_loss: 1.4819 - val_mean_squared_error: 1.4819
Epoch 4/20
1/1 [==============================] - 0s 55ms/step - loss: 1.2925 - mean_squared_error: 1.2925 - val_loss: 0.5078 - val_mean_squared_error: 0.5078
Epoch 5/20
1/1 [==============================] - 0s 60ms/step - loss: 1.8092 - mean_squared_error: 1.8092 - val_loss: 0.7207 - val_mean_squared_error: 0.7207
Epoch 6/20
1/1 [==============================] - 0s 43ms/step - loss: 2.9264 - mean_squared_error: 2.9264 - val_loss: 0.7077 - val_mean_squared_error: 0.7077
Epoch 7/20
1/1 [==============================] - 0s 41ms/step - loss: 2.7283 - mean_squared_error: 2.7283 - val_loss: 0.5246 - val_mean_squared_error: 0.5246
Epoch 8/20
1/1 [==============================] - 0s 46ms/step - loss: 1.7299 - mean_squared_error: 1.7299 - val_loss: 0.5316 - val_mean_squared_error: 0.5316
Epoch 9/20
1/1 [==============================] - 0s 46ms

([array([[-9.69162732e-02,  9.98411775e-02, -9.24931001e-03,
           1.00889830e-02,  1.02127478e-01,  3.92617770e-02,
           8.55163485e-02, -9.98023674e-02, -8.53581280e-02,
           1.86704770e-02,  5.80826662e-02, -1.10366963e-01,
          -6.62837178e-02,  6.60249069e-02,  4.21918416e-03,
           5.68390489e-02,  1.04435787e-01,  1.75418872e-02,
          -5.19082211e-02, -6.38228506e-02, -2.74068117e-02,
          -4.89139594e-02, -9.97123346e-02,  3.57695110e-02,
           1.07646741e-01, -9.62637737e-02,  1.15811788e-02,
          -4.06155847e-02, -2.21916921e-02, -7.41248354e-02,
          -1.44746313e-02, -6.02749959e-02,  7.80001953e-02,
          -9.58770588e-02,  2.17651539e-02,  3.83890457e-02,
          -9.15084872e-03, -5.73008880e-02,  1.03357034e-02,
           9.82197374e-02, -5.77835254e-02,  2.29794905e-02,
          -3.02039776e-02,  8.86730291e-03,  3.78361680e-02,
          -1.05710380e-01, -2.08937787e-02, -4.98018600e-02,
          -8.96593109e-0

In [11]:
import tensorflowjs as tfjs

In [13]:
init_weights = model.get_weights()
for i in range(3, 6):
    x, y = read_data(i)
    train(model, init_weights, x, y)
    tfjs.converters.save_keras_model(model, "./weights-{}".format(i))

Epoch 1/20
1/1 [==============================] - 0s 55ms/step - loss: 0.7148 - mean_squared_error: 0.7148 - val_loss: 1.6154 - val_mean_squared_error: 1.6154
Epoch 2/20
1/1 [==============================] - 0s 29ms/step - loss: 0.4048 - mean_squared_error: 0.4048 - val_loss: 2.0728 - val_mean_squared_error: 2.0728
Epoch 3/20
1/1 [==============================] - 0s 30ms/step - loss: 0.4297 - mean_squared_error: 0.4297 - val_loss: 2.3502 - val_mean_squared_error: 2.3502
Epoch 4/20
1/1 [==============================] - 0s 42ms/step - loss: 0.6054 - mean_squared_error: 0.6054 - val_loss: 2.3843 - val_mean_squared_error: 2.3843
Epoch 5/20
1/1 [==============================] - 0s 37ms/step - loss: 0.8544 - mean_squared_error: 0.8544 - val_loss: 2.1889 - val_mean_squared_error: 2.1889
Epoch 6/20
1/1 [==============================] - 0s 32ms/step - loss: 0.2182 - mean_squared_error: 0.2182 - val_loss: 1.9644 - val_mean_squared_error: 1.9644
Epoch 7/20
1/1 [==============================